In [1]:
import numpy as np
import matplotlib.pyplot as plt
import csv
import sys
import lmfit
from scipy import special
import function as func
import fit_script as fit

In [9]:
if __name__ == "__main__":
    #check_freq = np.array([18190, 18336, 19120, 19186, 19440, 19478, 19766, 19794, 19818, 20006, 20296, 20302, 20490, 20540, 20892, 21442, 21808, 22522, 22672, 23306, 23808, 23934, 25328, 25352, 25860, 26274, 26346])

    
    start_freq = 26.4
    print(start_freq)

    initial = int(start_freq * 1.e+6 - 250)
    final = int(initial + 1.e+5)
    
    
    for i in range():
        word = list(str(i*1000 - 250))
        word.insert(2, ".")
        start = "".join(word)
        print(start)

        signal = func.csv_to_array("/data/ms2840a/result_data/check_result/signal_12_data/start_{}GHz.csv".format(start))
        W1 = (signal["W_0"] + signal["W_1"] + signal["W_2"] + signal["W_3"] + signal["W_4"] + signal["W_5"] + signal["W_6"] + signal["W_7"] + signal["W_8"] + signal["W_9"] + signal["W_10"] + signal["W_11"])/12 

        signal = func.csv_to_array("/data/ms2840a/result_data/signal_12_data/start_{}GHz.csv".format(start))
        W2 = (signal["W_0"] + signal["W_1"] + signal["W_2"] + signal["W_3"] + signal["W_4"] + signal["W_5"] + signal["W_6"] + signal["W_7"] + signal["W_8"] + signal["W_9"] + signal["W_10"] + signal["W_11"])/12 

        
        freq, W, W_err = func.rebin_func(signal["freq"], (W1+W2/10)/1.1)

        # path for fit result on W1(additional measurement)+W2
        path = "/data/ms2840a/result_data/check_result/fit_result_mean_W/start_{}GHz.csv".format(start)

        fit.fitting(path, start, start_freq, freq, W)

    

18.189750
18.335750
19.119750
19.185750
19.439750
19.477750
19.765750
19.793750
19.817750
20.005750
20.295750
20.301750
20.489750
20.539750
20.891750
21.441750
21.807750
22.521750
22.671750
23.305750
23.807750
23.933750
25.327750
25.351750
25.859750
26.273750
26.345750
